In [1]:
import pandas as pd
import math
import matplotlib.pyplot as plt

In [2]:
def extract_chunk(df,idx,n_hist,n_pred,file,target,include_extra,lat_list,alt_list):
    valid = True
    chunk = None
    num_cities = df.shape[1]-1
    if (idx+n_hist+n_pred+1)*24 > df.shape[0]:
        valid = False
        return valid, chunk
    chunk = df.iloc[idx*24:(idx+n_hist)*24,:-1].T
    columns = []
    for i in range(n_hist-1,-1,-1):
        for j in range(12,12+24):
            columns.append(file+'_'+str(i)+'_'+str(j % 24))
    chunk.columns = columns
    if include_extra:
        day = df.iloc[idx*24:(idx+n_hist)*24,-1].reset_index(drop=True)
        chunk.insert(0,'day_of_year',day[0].day_of_year)
        chunk.insert(1,'latitude',lat_list)
        chunk.insert(1,'altitude',alt_list)
    if file == target:
        for i in range(n_pred):
            next_mean = df.iloc[(idx+n_hist+i)*24:(idx+n_hist+i+1)*24,:-1].T.mean(axis=1,skipna=False)
            chunk.insert(0,'mean_'+target+'_'+str(i+1),next_mean)
    chunk.index = range(idx*num_cities,(idx+1)*num_cities)
    return valid, chunk

In [3]:
def get_latitudes(city_list):
    df = pd.read_csv('city_attributes.csv')
    lat_list = []
    for city in city_list:
        lat_list.append(df.Latitude[df.City.str.find(city) == 0].values[0])
    return lat_list

In [4]:
n_pred = 7
n_hist = 1
target = 'humidity'
keep_idx = [-1,-2,-3,-4,-5,-6]
file_list = ['temperature','pressure','humidity','wind_direction','wind_speed']
city_list = ['Vancouver', 'Portland', 'San Francisco', 'Seattle', 'Los Angeles',
       'San Diego', 'Las Vegas', 'Phoenix', 'Albuquerque', 'Denver',
       'San Antonio', 'Dallas', 'Houston', 'Kansas City', 'Minneapolis',
       'Saint Louis', 'Chicago', 'Nashville', 'Indianapolis', 'Atlanta',
       'Detroit', 'Jacksonville', 'Charlotte', 'Miami', 'Pittsburgh',
       'Toronto', 'Philadelphia', 'New York', 'Montreal', 'Boston',
       'Beersheba', 'Tel Aviv District', 'Eilat', 'Haifa', 'Nahariyya',
       'Jerusalem']
altitudes = [0,50,52,174,305,62,2001,1086,5312,5279,650,430,79,909,830,466,597,
              597,719,738,656,16,761,6,1223,251,39,33,764,141,853,16,1148,33,46,2474]

keep_list = []
alt_list = []
for idx in keep_idx:
    keep_list.append(city_list[idx])
    alt_list.append(altitudes[idx])
lat_list = get_latitudes(keep_list)
keep_list.append('datetime')
master_df = pd.DataFrame()
for file in file_list:
    if file == target:
        include_extra = True
    else:
        include_extra = False
    df = pd.read_csv(file+'.csv')
    df = df.loc[:,keep_list]

    # changed = (df.iloc[:,1:].astype('float16') != df.iloc[:,1:]).sum().sum() - df.isna().sum().sum()
    temp = pd.to_datetime(df.datetime)
    # df = df.iloc[:,1:].astype('float16')
    # df['datetime'] = temp
    # print('Converting',file+'.csv to 16-bit fp\n\tNumber of changed values =',changed)
    ######################################
    df = df.drop('datetime',axis=1)
    df['datetime'] = temp

    gaps = 0
    for i in range(1,df.shape[0]):
        if (df.datetime[i].hour-df.datetime[i-1].hour) % 24 != 1:
            gaps = gaps + 1
    print('Checking',file+'.csv for missing values\n\t'+str(df.isna().sum().sum()),'NaNs\n\t'+str(gaps),'hour gaps')

    print('Checking',file+'.csv size\n\t'+str(df.shape[0]),'rows x',df.shape[1],'columns')
    
    valid = True
    big_chunk = pd.DataFrame()
    idx = 0;
    while valid:
        valid, chunk = extract_chunk(df,idx,n_hist,n_pred,file,target,include_extra,lat_list,alt_list)
        if big_chunk.shape[0] == 0:
            big_chunk = chunk
        elif valid:
            big_chunk = pd.concat([big_chunk,chunk])
        idx = idx + 1
    if master_df.shape[0] == 0:
            master_df = big_chunk
    else:
        master_df = master_df.join(big_chunk)

Checking temperature.csv for missing values
	4771 NaNs
	0 hour gaps
Checking temperature.csv size
	45253 rows x 7 columns
Checking pressure.csv for missing values
	4920 NaNs
	0 hour gaps
Checking pressure.csv size
	45253 rows x 7 columns
Checking humidity.csv for missing values
	5483 NaNs
	0 hour gaps
Checking humidity.csv size
	45253 rows x 7 columns
Checking wind_direction.csv for missing values
	4762 NaNs
	0 hour gaps
Checking wind_direction.csv size
	45253 rows x 7 columns
Checking wind_speed.csv for missing values
	4762 NaNs
	0 hour gaps
Checking wind_speed.csv size
	45253 rows x 7 columns


In [5]:
master_df.shape

(11262, 130)

In [6]:
master_df.dropna().shape

(10816, 130)

In [7]:
master_df = master_df.dropna()

In [8]:
master_df.to_hdf('big_data.h5','master_df',mode='w',format='table',complevel=9)

In [9]:
master_df.head(10)

,mean_temperature_7,mean_temperature_6,mean_temperature_5,mean_temperature_4,mean_temperature_3,mean_temperature_2,mean_temperature_1,day_of_year,altitude,latitude,...,wind_speed_0_2,wind_speed_0_3,wind_speed_0_4,wind_speed_0_5,wind_speed_0_6,wind_speed_0_7,wind_speed_0_8,wind_speed_0_9,wind_speed_0_10,wind_speed_0_11
3,299.906042,301.140417,299.718333,299.908125,300.634792,302.856458,302.795433,275,1148,29.558050,...,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0
6,296.587500,296.475000,297.020000,296.770833,297.652500,297.247500,301.199792,276,2474,31.769039,...,2.0,2.0,2.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0
7,298.190000,297.607500,298.117500,298.035417,298.867500,298.677500,300.139167,276,46,33.005859,...,6.0,6.0,6.0,6.0,3.0,3.0,3.0,3.0,3.0,3.0
8,298.190000,297.607500,298.117500,298.035417,298.867500,298.677500,300.197708,276,33,32.815559,...,6.0,6.0,6.0,6.0,3.0,3.0,3.0,3.0,3.0,3.0
9,298.917917,299.906042,301.140417,299.718333,299.908125,300.634792,302.856458,276,1148,29.558050,...,5.0,3.0,3.0,7.0,7.0,7.0,6.0,5.0,8.0,6.0
10,298.311250,298.098125,298.106667,298.641042,299.557083,300.056458,301.011250,276,16,32.083328,...,2.0,2.0,2.0,2.0,1.0,1.0,3.0,2.0,2.0,6.0
11,299.030000,298.639167,297.822917,299.481250,300.349167,299.457917,301.124375,276,853,31.251810,...,0.0,0.0,0.0,0.0,1.0,3.0,2.0,0.0,0.0,1.0
12,296.928125,296.587500,296.475000,297.020000,296.770833,297.652500,297.247500,277,2474,31.769039,...,3.0,3.0,3.0,4.0,3.0,3.0,3.0,3.0,3.0,3.0
13,297.966667,298.190000,297.607500,298.117500,298.035417,298.867500,298.677500,277,46,33.005859,...,5.0,5.0,5.0,5.0,4.0,4.0,4.0,4.0,4.0,4.0
14,297.966667,298.190000,297.607500,298.117500,298.035417,298.867500,298.677500,277,33,32.815559,...,5.0,5.0,5.0,5.0,4.0,4.0,4.0,4.0,4.0,4.0


In [10]:
for i in range(master_df.shape[1]):
    print(master_df.columns[i],master_df.iloc[0,i])

mean_temperature_7 299.9060416666666
mean_temperature_6 301.1404166666667
mean_temperature_5 299.7183333333333
mean_temperature_4 299.908125
mean_temperature_3 300.6347916666667
mean_temperature_2 302.85645833333336
mean_temperature_1 302.7954326922916
day_of_year 275
altitude 1148
latitude 29.55805
temperature_0_12 309.1
temperature_0_13 310.58
temperature_0_14 310.495769231
temperature_0_15 310.411538462
temperature_0_16 310.327307692
temperature_0_17 310.243076923
temperature_0_18 310.158846154
temperature_0_19 310.074615385
temperature_0_20 309.990384615
temperature_0_21 309.906153846
temperature_0_22 309.821923077
temperature_0_23 309.737692308
temperature_0_0 309.653461538
temperature_0_1 309.569230769
temperature_0_2 309.485
temperature_0_3 309.400769231
temperature_0_4 309.316538462
temperature_0_5 309.232307692
temperature_0_6 309.148076923
temperature_0_7 309.063846154
temperature_0_8 308.979615385
temperature_0_9 308.895384615
temperature_0_10 308.811153846
temperature_0_11 

In [11]:
df.columns

Index(['Jerusalem', 'Nahariyya', 'Haifa', 'Eilat', 'Tel Aviv District',
       'Beersheba', 'datetime'],
      dtype='object')